#### 数据说明

1、power_data文件夹下存放电力数据，government_data文件夹下存放政务数据。

2、电力数据包括：

- dlsj_df.csv：用户电费数据表
- dlsj_gl_2021.csv：用户功率数据表，每15分钟采集一次，每日96个功率数据
- dlsj_jcxx.csv：用户基础信息表
- dlsj_rydl.csv：用户日用电量表（复赛提供）

3、政务数据包括：

- dj_nsrxx_kz.csv：工商快照表（复赛提供）

- sb_xssr.csv：销售收入表

- sz_tmp_baseinfo_ent.csv：纳税人扩展信息表

- zs_jks.csv：缴款书表，此表中的SJJE（实缴金额）可用来计算每家公司的单季度累计税收


4、缴款书表中，若存在跨越季度的缴税记录，如税款所属期起是1月，税款所属期止是5月，则需要将该条记录的实缴金额SJJE除以月份数，将其金额均摊到每个月份中，再计算每季度的累计税额。



In [9]:
import pandas as pd
import numpy as np

from pathlib import Path
base_dir = Path.cwd() / '初赛数据集'


In [18]:
# 缴款书表
jks_file = base_dir / 'train' / 'government_data' / 'zs_jks缴款书表.csv'
jks_df = pd.read_csv(jks_file, parse_dates=['税款所属期起', '税款所属期止'])[['公司ID', '税款所属期起', '税款所属期止', '实缴金额']]
jks_df = jks_df.rename(columns = {
    '公司ID': 'ID',
    '税款所属期起': 'start_date',
    '税款所属期止': 'end_date',
    '实缴金额': 'sjje'
})
print(len(jks_df['ID'].unique()))
# jks_df.groupby('start_date').count().to_csv('start_date_count.csv')
jks_df[:20]

500


,ID,start_date,end_date,sjje
0,2022865.0,2019-06-01,2019-06-30,1332
1,2022865.0,2019-01-01,2019-01-31,46
2,2022865.0,2019-03-01,2019-03-31,15
3,2022865.0,2019-01-01,2019-03-31,17662
4,2022865.0,2019-03-01,2019-03-31,30
5,2022865.0,2019-02-01,2019-02-28,100
6,2022865.0,2019-02-01,2019-02-28,31
7,2022865.0,2019-01-01,2019-01-31,30
8,2022865.0,2018-10-01,2018-12-31,10039
9,2022865.0,2018-10-01,2018-12-31,7850


In [19]:
jks_df['start_date'] = jks_df['start_date'].to_numpy().astype('datetime64[M]')
jks_df['end_date'] = jks_df['end_date'].to_numpy().astype('datetime64[M]')
jks_df['date'] = jks_df.apply(lambda row: pd.date_range(start=row['start_date'], end=row['end_date'], freq='MS'), axis=1)
jks_df['date'] = jks_df['date'].map(lambda x: [str(d.date()) for d in x])

In [20]:
jks_df['date_len'] = jks_df['date'].map(lambda x: len(x))
jks_df['sjje_per_month'] = jks_df['sjje'] / jks_df['date_len']
jks_df

,ID,start_date,end_date,sjje,date,date_len,sjje_per_month
0,2022865.0,2019-06-01,2019-06-01,1332,[2019-06-01],1,1332.000000
1,2022865.0,2019-01-01,2019-01-01,46,[2019-01-01],1,46.000000
2,2022865.0,2019-03-01,2019-03-01,15,[2019-03-01],1,15.000000
3,2022865.0,2019-01-01,2019-03-01,17662,"[2019-01-01, 2019-02-01, 2019-03-01]",3,5887.333333
4,2022865.0,2019-03-01,2019-03-01,30,[2019-03-01],1,30.000000
...,...,...,...,...,...,...,...
228851,2022153.0,2021-10-01,2021-12-01,264085,"[2021-10-01, 2021-11-01, 2021-12-01]",3,88028.333333
228852,2022153.0,2021-10-01,2021-12-01,30730,"[2021-10-01, 2021-11-01, 2021-12-01]",3,10243.333333
228853,2022153.0,2021-12-01,2021-12-01,71,[2021-12-01],1,71.000000
228854,2022153.0,2021-10-01,2021-12-01,7836,"[2021-10-01, 2021-11-01, 2021-12-01]",3,2612.000000


In [21]:
jks_df_per_month = jks_df.explode('date')
jks_df_per_month['date'] = jks_df_per_month['date'].to_numpy().astype('datetime64[M]').astype(str)
# print(jks_df_per_month.head())
print(jks_df.shape, np.sum(jks_df['date_len']*jks_df['sjje_per_month']), np.sum(jks_df['sjje']))
np.sum(jks_df_per_month['sjje_per_month'])
print(jks_df_per_month.shape, np.sum(jks_df['date_len']), np.sum(jks_df['sjje']), np.sum(jks_df_per_month['sjje_per_month']))
# jks_df_per_month.to_csv('a.csv')
# jks_df.to_csv('b.csv')
jks_df_per_month_group = jks_df_per_month.groupby(['ID', 'date'])['sjje_per_month'].sum()
# jks_df_per_month_group.groupby('ID').count().to_csv('ID_count.csv')

jks_df_per_month_group.to_csv('output/per_month.csv')


(228856, 7) 42834560915.0 42834560915
(393540, 7) 393540 42834560915 42834560915.0


##### 销售收入表

In [22]:
xssr_file = base_dir / 'train' / 'government_data' / 'sb_xssr销售收入表.csv'
xssr_df = pd.read_csv(xssr_file, parse_dates=['申报日期'])[
    ['公司ID', '本月数、本期数(按季申报)', '本年累计', '申报日期']
]
xssr_df = xssr_df.rename(columns={
    '公司ID': 'ID',
    '本月数、本期数(按季申报)': 'month_amount',
    '本年累计': 'year_amount',
    '申报日期': 'date'
})
xssr_df['date'] = xssr_df['date'].to_numpy().astype('datetime64[M]').astype(str)
# print(len(xssr_df['ID'].unique()))
xssr_df = xssr_df.groupby(['ID', 'date']).agg({'month_amount' : 'mean', 'year_amount' : 'mean'})
xssr_df

month_amount   year_amount
ID      date                               
2022002 2018-01    97440295.0  1.238262e+09
        2018-02    97732400.0  9.773240e+07
        2018-03    60087323.0  1.578197e+08
        2018-04    98816315.0  2.566360e+08
        2018-05   112146711.0  3.687827e+08
...                       ...           ...
2022917 2021-08     4402361.0  3.411071e+07
        2021-09     4255877.0  3.836659e+07
        2021-10     4528709.0  4.289530e+07
        2021-11     4413717.0  4.730901e+07
        2021-12     6165673.0  5.347468e+07

[23186 rows x 2 columns]

#### 企业信息

In [23]:
biz_file = base_dir / 'train' / 'government_data' / 'sz_tmp_baseinfo_ent纳税人扩展信息表.csv'
biz_df = pd.read_csv(biz_file)[
    ['公司ID', '注册资本金（万元）']
]

biz_df = biz_df.rename(columns={
    '公司ID': 'ID',
    '组织': 'zuzhi',
    '批准设立机关': 'jiguan',
    '注册资本金（万元）': 'zijin'
})
biz_df['zijin'] = biz_df['zijin'].map(lambda x: x.replace('万', '')).fillna(0)
# print(len(biz_df['ID'].unique()))
biz_df

,ID,zijin
0,2022865,3000
1,2022488,7172.41
2,2022058,100
3,2022063,2500
4,2022885,500
...,...,...
511,2022097,18000
512,2022087,25000
513,2022338,0
514,2022796,1058


#### merge data

In [24]:
new_df = pd.merge(jks_df_per_month_group, xssr_df, on=['ID', 'date'])
new_df.to_csv('merge.csv')
print(new_df.shape, jks_df_per_month_group.shape, xssr_df.shape)
all_df = pd.merge(new_df, biz_df, how='left', on='ID')
print(all_df.shape)
all_df.to_csv('government_train.csv')

(22997, 3) (28465,) (23186, 2)
(23762, 5)


In [28]:
df_templ = pd.DataFrame(jks_df['ID'].unique(), columns=['ID'])
df_templ.loc[:, 'date'] = [[str(d.date()) for d in pd.date_range(start='2019-1-1', end='2021-12-1', freq='MS')]] * len(df_templ)
df_templ = df_templ.explode('date')
print(df_templ.shape)
df_templ['date'] = df_templ['date'].to_numpy().astype('datetime64[M]').astype(str)
merge_jks = pd.merge(df_templ, jks_df_per_month_group, on=['ID', 'date'], how='left')

merge_jks = pd.merge(merge_jks, xssr_df, on=['ID', 'date'], how='left')
print(merge_jks.shape)
# merge_jks = merge_jks.merge(biz_df, how='left', on='ID', indicator=True)
# print(merge_jks.shape)
merge_jks = merge_jks.sort_values(['ID', 'date']).reset_index().drop(columns=['index'])
merge_jks['month_amount'] = merge_jks['month_amount'].fillna(merge_jks.groupby('ID')['month_amount'].transform('mean'))
merge_jks['year_amount'] = merge_jks['year_amount'].fillna(merge_jks.groupby('ID')['year_amount'].transform('mean'))
merge_jks['sjje_per_month'] = merge_jks['sjje_per_month'].fillna(merge_jks.groupby('ID')['sjje_per_month'].transform('mean'))
# merge_jks.to_csv('merge_jks.csv')

merge_jks['month_amount2'] = np.square(merge_jks['month_amount'])
merge_jks['year_amount2'] = np.square(merge_jks['year_amount'])

# 日期取做年、月，删除店铺ID，其余字段归一化
# merge_jks.loc[:, ['year', 'month']] = merge_jks.apply(
#     lambda row: pd.Series({'year': row['date'].split('-')[0], 'month': row['date'].split('-')[1]}), axis=1
#     )
merge_jks['year'] = merge_jks['date'].map(lambda x: x.split('-')[0])
merge_jks['month'] = merge_jks['date'].map(lambda x: x.split('-')[1])
gover_data = merge_jks.drop(columns=['ID', 'date'])
# gover_data['month_amount'] = gover_data['month_amount'] / gover_data['month_amount'].mean()
# gover_data['year_amount'] = gover_data['year_amount'] / gover_data['year_amount'].mean()
gover_data.index.rename('idx', inplace=True)

gover_data.to_csv('output/gover_data.csv')


(18000, 2)
(18000, 5)


#### 电力数据

In [30]:
dlsj_file = base_dir / 'train' / 'power_data' / 'dlsj_df用户电费数据表.csv'

dlsj_df = pd.read_csv(dlsj_file, parse_dates=['应收年月'])[['公司ID', '应收年月', '能源量', '应收金额']]
dlsj_df = dlsj_df.rename(columns={
    '公司ID': 'ID',
    '应收年月': 'date',
    '能源量': 'nyl',
    '应收金额': 'ysje'
})
dlsj_df['date'] = pd.to_datetime(dlsj_df['date'], format="%Y%m").to_numpy().astype('datetime64[M]').astype(str)
# dlsj_df = dlsj_df['date'].unique()
# dlsj_df['date'] = dlsj_df['date'].to_numpy().astype('datetime64[M]').astype(str)
dlsj_df = dlsj_df.groupby(['ID', 'date']).agg(
     nyl = ('nyl','sum'),
     ysje = ('ysje','sum'),
     )
dlsj_df

df_templ = pd.DataFrame(jks_df['ID'].unique(), columns=['ID'])
df_templ.loc[:, 'date'] = [[str(d.date()) for d in pd.date_range(start='2019-1-1', end='2021-12-1', freq='MS')]] * len(df_templ)
df_templ = df_templ.explode('date')
df_templ['date'] = df_templ['date'].to_numpy().astype('datetime64[M]').astype(str)

merge_dlsj = pd.merge(df_templ, dlsj_df, on=['ID', 'date'], how='left').sort_values(['ID', 'date']).reset_index().drop(columns=['index'])
merge_dlsj['nyl'] = merge_dlsj['nyl'].fillna(merge_dlsj.groupby('ID')['nyl'].transform('mean'))
merge_dlsj['ysje'] = merge_dlsj['ysje'].fillna(merge_dlsj.groupby('ID')['ysje'].transform('mean'))

merge_dlsj['nyl2'] = np.square(merge_dlsj['nyl'])
merge_dlsj['ysje2'] = np.square(merge_dlsj['ysje'])

merge_dlsj['year'] = merge_dlsj['date'].map(lambda x: x.split('-')[0])
merge_dlsj['month'] = merge_dlsj['date'].map(lambda x: x.split('-')[1])
merge_dlsj[['ID', 'date']].to_csv('output/ID_date_templ.csv')
power_data = merge_dlsj.drop(columns=['ID', 'date'])
power_data.index.rename('idx', inplace=True)
power_data.to_csv('output/power_data.csv')